In [1]:
from main import launch_demo

In [2]:
launch_demo()

🎯 Launching Master Thesis Demo Application
🔍 Found available ports - API: 8000, Client: 8080
✅ CORS configured for origins: ['http://localhost:8080', 'http://127.0.0.1:8080', '*']
📡 Starting API server on 0.0.0.0:8000


INFO:     Started server process [15796]
INFO:     Waiting for application startup.


2025-09-03 23:58:36,951 - root - INFO - Logging configured with level INFO


INFO:     Application startup complete.
ERROR:    [Errno 10048] error while attempting to bind on address ('0.0.0.0', 8000): only one usage of each socket address (protocol/network address/port) is normally permitted
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.


✅ API server is ready at http://localhost:8000
🚀 Starting client application on port 8080...
⏳ Waiting for client to start...

✅ Demo application started successfully!
📡 API: http://localhost:8000
🌐 Client: http://localhost:8080
💡 If ports were busy, alternative ports were automatically selected
